In [38]:
def project_2_scoring(data):
    
    from copy import deepcopy
    from sklearn.preprocessing import LabelEncoder
    import category_encoders as ce
    from sklearn.model_selection import StratifiedKFold
    import pickle
    import numpy as np
    
    X = data.copy()
    
    '''Load Artifacts'''
    artifacts_dict_file = open("artifacts_proj2_file.pkl", "rb")
    artifacts_dict = pickle.load(file=artifacts_dict_file)
    artifacts_dict_file.close()
    
    model = artifacts_dict["model"],
    woe_cols = artifacts_dict["woe_cols"]
    woe_encoder = artifacts_dict["woe_encoder"]
    ohe_cols = artifacts_dict["ohe_columns"]
    le = artifacts_dict["label_encoder"]
    drop_from_data = artifacts_dict["drop_from_data"]
    col_remove_dollar = artifacts_dict["col_remove_dollar"]
    threshold = artifacts_dict["threshold"]
    original_columns= artifacts_dict["original_columns"]
    parameters=artifacts_dict["columns_to_score"]
    
   
    fill_na={}
    for col in X.columns:
        if pd.api.types.is_numeric_dtype(X[col].dtype):
            fill_na[col]=0
        else:
            fill_na[col]='Missing'
    X.fillna(value=fill_na,inplace=True)
    
    # Remove all the columns that were unseen during training
    X=X[original_columns]
    # Remove Dollars from numerical vars
    for col in col_remove_dollar:
        X[col] = X[col].str.replace('$','',regex=True).replace(',','',regex=True)
        X[col] = X[col].astype(float)
    
    # Adding New Features
    X['NAICS'] = X['NAICS']%100
    X['NAICS'] = X['NAICS'].astype('category',copy=False)
    
    X['Net_Loan']=X['DisbursementGross']-X['BalanceGross']
    X['SBA_Payable']=(X['SBA_Appv']/X['GrAppv'])
    X['AppvGross']=(X['GrAppv']*X['DisbursementGross'])
    X['Jobs_Loan']=(X['CreateJob']*X['GrAppv'])
    
    X['LowDoc']=X['LowDoc'].astype('category',copy=True)
    
    
    X['NewExist']=X['NewExist'].astype('category',copy=True)
    X = X.loc[X['NewExist'] != 0]
    X['NewExist_encoded'] = le.fit_transform(X['NewExist'])
    X['loan_history'] = X['DisbursementGross'] * X['NewExist_encoded']
    X=X.drop(columns=['NewExist_encoded'])
    
    X['UrbanRural']=X['UrbanRural'].apply(lambda x: 'Urban' if x == 1 else  'Rural' if x==2 else 'Undefined')
    X['UrbanRural']=X['UrbanRural'].astype('category',copy=True)
    X['UrbanRural_encoded'] = le.fit_transform(X['UrbanRural'])
    X['loan_history'] = X['DisbursementGross'] * X['UrbanRural_encoded']
    X=X.drop(columns=['UrbanRural_encoded'])
    
    X['FranchiseCode']=X['FranchiseCode'].apply(lambda x: 'N' if x in [0,1] else 'Y')
    X['FranchiseCode'] = X['FranchiseCode'].astype('category',copy=False)
    X['LoanLimit']=(X['LoanInd']*X['DisbursementGross'])

    #one hot encoding
    for col in ohe_cols:
        one_hot_enc_X=pd.get_dummies(X[col], prefix=col)
        X = pd.concat([X, one_hot_enc_X], axis=1)
    
    # WOE Encoding        
    woe_enc_test = woe_encoder.transform(X[woe_cols])
    for col in woe_cols:
        X[col+"_woe"] = woe_enc_test[col]
    
      
    # logistic regression predict
    X_test=X[parameters]
    model=artifacts_dict['model']
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba >= threshold).astype(int)
    d = {"label":     y_pred ,
     "probability_0": (1-y_pred_proba),
     "probability_1": y_pred_proba}
    
    return pd.DataFrame(d)

In [39]:
import pandas as pd
#loan = pd.read_csv("test_data.csv")
#loan1=loan.drop(columns=['MIS_Status']).head(10000)
#project_2_scoring(loan1)

[4.85356317e-04 2.26626093e-01 2.41285123e-01 ... 5.73620865e-01
 2.70342393e-01 6.88096132e-01] [0.99951464 0.77337391 0.75871488 ... 0.42637914 0.72965761 0.31190387]


,label,probability_0,probability_1
0,0,0.999515,0.000485
1,0,0.773374,0.226626
2,0,0.758715,0.241285
3,0,0.835951,0.164049
4,0,0.738573,0.261427
...,...,...,...
6046,0,0.970524,0.029476
6047,0,0.787975,0.212025
6048,0,0.426379,0.573621
6049,0,0.729658,0.270342
